In [1]:
import sys
print(sys.version)

3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]


In [2]:
import os
import numpy as np
import torch
from PIL import Image
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Convert images to tensors and add labels

real_cats_dir = '../data/aesthetic_6k/cat_aesthetic'
generated_cats_dir = '../data/cat_generated_aesthetic'
convert_tensor = transforms.ToTensor()

def get_images(dir, label):
    images = []
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f):
            img = Image.open(f)
            images.append((convert_tensor(img), label))
    return images

# label 0 indicates real image
real_images = get_images(real_cats_dir, 0)
# label 1 indicates AI image
generated_images = get_images(generated_cats_dir, 1)

In [4]:
# Combine real and generated for full dataset
# Split into training and testing sets

full_dataset = real_images + generated_images

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
images, labels = iter(trainloader).next()

In [5]:
class FCNN(nn.Module):
  def __init__(self, input, hiddensize = 100):
    super().__init__()  # needed to invoke the properties of the parent class nn.Module
    self.in_layer = nn.Linear(input, hiddensize)
    self.out_layer = nn.Linear(hiddensize, 10)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.in_layer(x))
    x = self.out_layer(x)
    return x


## Training

In [6]:
# images.shape: torch.Size([16, 3, 256, 256])

epochs = 15
input_size = 3*256*256
hidden_size = 100
model = FCNN(input_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3) 
train_mse = []
# training iterations
for epoch in range(epochs):
    running_loss = 0
    for itr, (image, label) in enumerate(trainloader):
        # Flatten MNIST images into a 784 long vector
        image = image.view(image.shape[0], -1)
        # zero gradient
        optimizer.zero_grad()
        # forward path
        y_predicted = model(image)
        loss = criterion(y_predicted, label)
        running_loss += loss.item()
        # backpropagating
        loss.backward()
        # optimizes the weights
        optimizer.step()
    train_mse.append(running_loss)
    if (epoch+1) % 3 == 0:
        print(f'epoch: {epoch+1}, loss: {running_loss:.4f}')

epoch: 3, loss: 430.8718
epoch: 6, loss: 396.3283
epoch: 9, loss: 366.3928
epoch: 12, loss: 338.2157
epoch: 15, loss: 314.8653


In [7]:
correct = 0
total = 0
with torch.no_grad():
    for itr, (image, label) in enumerate(testloader):
        image = image.view(image.shape[0], -1)
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
        correct += predicted.eq(label.reshape(len(label),)).sum() 
        total += float(len(label))
    accuracy = correct / total
    print(f'Accuracy of Neural Network is {accuracy:.4f}')

Accuracy of Neural Network is 0.7129
